In [ ]:
#implementing hsv+contour+gaussian blur on pic

In [ ]:
#algo

In [ ]:
# copy image into another varibale for later use
# adjusting color space using hsv
# using blur to reduce noise
# apply mask using hsv
# canny edge 
# dilate
# call contour function
# display

In [5]:
import cv2
import numpy as np


def nothing(x):
    pass

cv2.namedWindow("Tracking")
cv2.createTrackbar("LH", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LS", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LV", "Tracking", 0, 255, nothing)
cv2.createTrackbar("UH", "Tracking", 255, 255, nothing)
cv2.createTrackbar("US", "Tracking", 255, 255, nothing)
cv2.createTrackbar("UV", "Tracking", 255, 255, nothing)

#cv2.namedWindow("Parameters")
#cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Tracking",107,255,nothing)
cv2.createTrackbar("Threshold2","Tracking",114,255,nothing)
cv2.createTrackbar("Area","Tracking",5000,30000,nothing)

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #in the last parameter we are getting all the stored contour points but if we use the other one which is cv2.CHAIN_APPROX_SIMPLE then this will compress the values and we will get lesser number of points so we write it as NONE so that we get all of the points.
    #in external it retrives only the external outer corners
    #cv2.drawContours(imgContour, contours, -1, (255, 0, 255), 7)#last 2 parameters is colour and width of boundary
    
    for cnt in contours:
        #now we are looking into the area of each contour. so basically by putting a limit on the area we are reducing the noise like small dots outside the object
        #cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
        area=cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Tracking")
        if area>areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            #now finding the corner points but before that we need to find the length of the contours
            peri=cv2.arcLength(cnt,True)#true basically means that the contour is closed and this will give us the value of the length of the perimeter and we will use the perimeter to approximate what type of shape this is.
            #we will use the perimeter to approximate what type of shape this is.
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)#giving it resolution and then defining that this is a closed contour. This approximation array will have a certain amount of points and these points can determine whether its a square or any other shape.
            #This approximation array will have a certain amount of points and these points can determine whether its a square or any other shape.
            print(len(approx)) #printing out the number of points. based on the number of points we can later generalise the shape.
            #now lets create a bounding box
            x, y, w, h=cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5) #2nd arg is initial points. 3rd argument is the final corner of the rectangle or the bounding box.after that color followed by width
            #now we are going to display our values so that we can easily see the number of points detected and the area detected.
            cv2.putText(imgContour, "Points: " + str(len(approx)), (x, y - 42), cv2.FONT_HERSHEY_COMPLEX, .7,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x, y-27), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            
            
while True:
    frame = cv2.imread('apple (281).jpg')
    imgContour = frame.copy()
    imgBlur = cv2.GaussianBlur(frame, (7, 7), 1)#using kernel of 7 by 7 and 1 iteration

    hsv = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("LH", "Tracking")
    l_s = cv2.getTrackbarPos("LS", "Tracking")
    l_v = cv2.getTrackbarPos("LV", "Tracking")

    u_h = cv2.getTrackbarPos("UH", "Tracking")
    u_s = cv2.getTrackbarPos("US", "Tracking")
    u_v = cv2.getTrackbarPos("UV", "Tracking")
    
    l_b=np.array([l_h,l_s,l_v])
    u_b=np.array([u_h,u_s,u_v])
    
    mask=cv2.inRange(hsv, l_b, u_b)
    
    res=cv2.bitwise_and(frame, frame,mask=mask)
    
    threshold1 = cv2.getTrackbarPos("Threshold1", "Tracking")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Tracking")
    imgCanny = cv2.Canny(mask,threshold1,threshold2)
    kernel = np.ones((5, 5)) #defining a kernel for the dilation function. 
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)#dilation function is used to remove noise from the image.
    getContours(imgDil,imgContour)
    
    imgStack = stackImages(0.8,([frame,mask,res],
                               [imgCanny,imgDil,imgContour]))
    
    cv2.imshow('result',imgStack)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    
    
    '''ret, thresh = cv2.threshold(mask, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)#2nd arg is contour retrieval mode,contour approxiation method
    #hierarchy contains information of image topology
    #RETR_EXTERNAL retrives only the external outer corners
    #TREE retrives all the contours and constructs a full hierarchy
    print("Number of contours = " + str(len(contours)))#this will give the number of contours in the image
    print(contours[0])

    cv2.drawContours(mask, contours, -1, (0, 255, 0), 3)#3rd arg is contour indexes. if we give that -1 than its going to find all the 9 contours in the image. if we give index 0 then first contour will be found out in the image.similarly u can go from 0 to 8 as total contours are 9

    cv2.drawContours(res, contours, -1, (0, 255, 0), 3)
    
    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('result',res)
    
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break'''
        
cv2.destroyAllWindows()

In [ ]:
# apply morph in above code and try

In [ ]:
# implementing hsv+contour+gaussian blur in real time

In [1]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

def nothing(x):
    pass

cv2.namedWindow("Tracking")
cv2.createTrackbar("LH", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LS", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LV", "Tracking", 0, 255, nothing)
cv2.createTrackbar("UH", "Tracking", 255, 255, nothing)
cv2.createTrackbar("US", "Tracking", 255, 255, nothing)
cv2.createTrackbar("UV", "Tracking", 255, 255, nothing)

#cv2.namedWindow("Parameters")
#cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Tracking",107,255,nothing)
cv2.createTrackbar("Threshold2","Tracking",114,255,nothing)
cv2.createTrackbar("Area","Tracking",5000,30000,nothing)

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #in the last parameter we are getting all the stored contour points but if we use the other one which is cv2.CHAIN_APPROX_SIMPLE then this will compress the values and we will get lesser number of points so we write it as NONE so that we get all of the points.
    cv2.drawContours(imgContour, contours, -1, (255, 0, 255), 7)#last 2 parameters is colour and width of boundary

    for cnt in contours:
        cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
        area=cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Tracking")
        if area>areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            #now finding the corner points but before that we need to find the length of the contours
            peri=cv2.arcLength(cnt,True)#true basically means that the contour is closed and this will give us the value of the length of the perimeter and we will use the perimeter to approximate what type of shape this is.
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)#giving it resolution and then defining that this is a closed contour. This approximation array will have a certain amount of points and these points can determine whether its a square or any other shape.
            print(len(approx))
            #now lets create a bounding box
            x, y, w, h=cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5) #2nd arg is initial points. 3rd argument is the final corner of the rectangle or the bounding box.after that color followed by width
            #now we are going to display our values so that we can easily see the number of points detected and the area detected.
            cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, .7,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "fruit detected: Tomato ", (x + w + 20, y + 70), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            
            
while True:
    success, frame = cap.read()
    #frame = cv2.imread('apple (281).jpg')
    imgContour = frame.copy()
    imgBlur = cv2.GaussianBlur(frame, (7, 7), 1)#using kernel of 7 by 7

    hsv = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("LH", "Tracking")
    l_s = cv2.getTrackbarPos("LS", "Tracking")
    l_v = cv2.getTrackbarPos("LV", "Tracking")

    u_h = cv2.getTrackbarPos("UH", "Tracking")
    u_s = cv2.getTrackbarPos("US", "Tracking")
    u_v = cv2.getTrackbarPos("UV", "Tracking")
    
    l_b=np.array([l_h,l_s,l_v])
    u_b=np.array([u_h,u_s,u_v])
    
    mask=cv2.inRange(hsv, l_b, u_b)
    
    res=cv2.bitwise_and(frame, frame,mask=mask)
    
    threshold1 = cv2.getTrackbarPos("Threshold1", "Tracking")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Tracking")
    imgCanny = cv2.Canny(mask,threshold1,threshold2)
    kernel = np.ones((5, 5)) #defining a kernel for the dilation function. 
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)#dilation function is used to remove noise from the image.
    getContours(imgDil,imgContour)
    
    imgStack = stackImages(0.8,([frame,mask,res],
                               [imgDil,imgCanny,imgContour]))
    
    cv2.imshow('result',imgStack)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    
    
    '''ret, thresh = cv2.threshold(mask, 127, 255, 0)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)#2nd arg is contour retrieval mode,contour approxiation method
    #hierarchy contains information of image topology
    #RETR_EXTERNAL retrives only the external outer corners
    #TREE retrives all the contours and constructs a full hierarchy
    print("Number of contours = " + str(len(contours)))#this will give the number of contours in the image
    print(contours[0])

    cv2.drawContours(mask, contours, -1, (0, 255, 0), 3)#3rd arg is contour indexes. if we give that -1 than its going to find all the 9 contours in the image. if we give index 0 then first contour will be found out in the image.similarly u can go from 0 to 8 as total contours are 9

    cv2.drawContours(res, contours, -1, (0, 255, 0), 3)
    
    cv2.imshow('frame',frame)
    cv2.imshow('mask',mask)
    cv2.imshow('result',res)
    
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break'''
cap.release()        
cv2.destroyAllWindows()

10
10
10
9
8
11
11
13
9
13
12
9
9
13
11
9
10
11
10
11
10
10
10
10
13
12
12
9
7
11
13
12
12
11
9
12
9
9
11
10
9
13
12
14
11
11
11
11
10
10
9
11
11
10
11
10
10
9
10
9
10
12
11
11
12
10
11
8
8
10
10
10
10
10
9
11
11
13
11
10
9
11
10
10
9
11
11
11
10
10
10
14
14
11
11
11
13
9
10
12
14
12
12
11
12
10
11
12
12
12
9
10
10
9
12
10
9
11
10
10
11
11
10
10
13
8
10
12
8
12
10
10
8
11
10
9
12
12
10
12
9
10
8
11
10
11
11
9
12
9
10
9
10
12
10
12
9
11
11
10
10
9
10
10
10
10
9
10
14
10
12
12
10
10
9
12
10
13
11
11
10
9
8
10
8
8
12
10
8
10
11
10


In [ ]:
# hsv+contour+median blur in real time

In [12]:
import cv2
import numpy as np

frameWidth = 640
frameHeight = 480
cap = cv2.VideoCapture(0)
cap.set(3, frameWidth)
cap.set(4, frameHeight)

def nothing(x):
    pass

cv2.namedWindow("Tracking")
cv2.createTrackbar("LH", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LS", "Tracking", 0, 255, nothing)
cv2.createTrackbar("LV", "Tracking", 0, 255, nothing)
cv2.createTrackbar("UH", "Tracking", 255, 255, nothing)
cv2.createTrackbar("US", "Tracking", 255, 255, nothing)
cv2.createTrackbar("UV", "Tracking", 255, 255, nothing)

#cv2.namedWindow("Parameters")
#cv2.resizeWindow("Parameters",640,240)
cv2.createTrackbar("Threshold1","Tracking",107,255,nothing)
cv2.createTrackbar("Threshold2","Tracking",114,255,nothing)
cv2.createTrackbar("Area","Tracking",5000,30000,nothing)

def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #in the last parameter we are getting all the stored contour points but if we use the other one which is cv2.CHAIN_APPROX_SIMPLE then this will compress the values and we will get lesser number of points so we write it as NONE so that we get all of the points.
    cv2.drawContours(imgContour, contours, -1, (255, 0, 255), 7)#last 2 parameters is colour and width of boundary

    for cnt in contours:
        cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
        area=cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Tracking")
        if area>areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            #now finding the corner points but before that we need to find the length of the contours
            peri=cv2.arcLength(cnt,True)#true basically means that the contour is closed and this will give us the value of the length of the perimeter and we will use the perimeter to approximate what type of shape this is.
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)#giving it resolution and then defining that this is a closed contour. This approximation array will have a certain amount of points and these points can determine whether its a square or any other shape.
            print(len(approx))
            #now lets create a bounding box
            x, y, w, h=cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5) #2nd arg is initial points. 3rd argument is the final corner of the rectangle or the bounding box.after that color followed by width
            #now we are going to display our values so that we can easily see the number of points detected and the area detected.
            cv2.putText(imgContour, "Points: " + str(len(approx)), (x + w + 20, y + 20), cv2.FONT_HERSHEY_COMPLEX, .7,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x + w + 20, y + 45), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "fruit detected: Tomato ", (x + w + 20, y + 70), cv2.FONT_HERSHEY_COMPLEX, 0.7,
                        (0, 255, 0), 2)
            
            
while True:
    success, frame = cap.read()
    #frame = cv2.imread('apple (281).jpg')
    imgContour = frame.copy()
    imgBlur = cv2.medianBlur(frame, 7)#using kernel of 7 by 7

    hsv = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    
    l_h = cv2.getTrackbarPos("LH", "Tracking")
    l_s = cv2.getTrackbarPos("LS", "Tracking")
    l_v = cv2.getTrackbarPos("LV", "Tracking")

    u_h = cv2.getTrackbarPos("UH", "Tracking")
    u_s = cv2.getTrackbarPos("US", "Tracking")
    u_v = cv2.getTrackbarPos("UV", "Tracking")
    
    l_b=np.array([l_h,l_s,l_v])
    u_b=np.array([u_h,u_s,u_v])
    
    mask=cv2.inRange(hsv, l_b, u_b)
    
    res=cv2.bitwise_and(frame, frame,mask=mask)
    
    threshold1 = cv2.getTrackbarPos("Threshold1", "Tracking")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Tracking")
    imgCanny = cv2.Canny(mask,threshold1,threshold2)
    kernel = np.ones((5, 5)) #defining a kernel for the dilation function. 
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)#dilation function is used to remove noise from the image.
    getContours(imgDil,imgContour)
    
    imgStack = stackImages(0.8,([frame,mask,res],
                               [imgDil,imgCanny,imgContour]))
    
    cv2.imshow('median blur',imgBlur)
    cv2.imshow('result',imgStack)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
    
cap.release()        
cv2.destroyAllWindows()

In [4]:
import cv2
import numpy as np

img = cv2.imread('lemons.jpeg')
img1 = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img1 = cv2.medianBlur(img1,1)
# cv2.imshow("Blur",img1)
# cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)

circles=cv2.HoughCircles(img1,cv2.HOUGH_GRADIENT,1, 30, np.array([]), 80, 20, 3, 50)

circles = np.uint16(np.around(circles))
for i in circles[0,:]:
    
    cv2.circle(img,(i[0],i[1]),i[2],(0,255,0),2)
    
font = cv2.FONT_HERSHEY_SIMPLEX
cv2.putText(img,str(circles.shape[1])+' lemons detected',(10,50), font, 0.8,(255,255,255),2,cv2.LINE_AA)
cv2.imshow('detected lemons',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#merging two codes- pic version (trial 1)

In [ ]:

#im to gray
#mblur
#split
#blur
#th1, blur,thresh_bin
#closing morph, k1
#opening morph, k2
#laplacian

In [2]:
import cv2
import numpy as np

def nothing(x):
    pass

cv2.namedWindow("Tracking")
cv2.createTrackbar("Area","Tracking",5000,30000,nothing)

def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) #in the last parameter we are getting all the stored contour points but if we use the other one which is cv2.CHAIN_APPROX_SIMPLE then this will compress the values and we will get lesser number of points so we write it as NONE so that we get all of the points.
    cv2.drawContours(imgContour, contours, -1, (255, 0, 255), 7)#last 2 parameters is colour and width of boundary

    for cnt in contours:
        cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
        area=cv2.contourArea(cnt)
        areaMin = cv2.getTrackbarPos("Area", "Tracking")
        if area>areaMin:
            cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            #now finding the corner points but before that we need to find the length of the contours
            peri=cv2.arcLength(cnt,True)#true basically means that the contour is closed and this will give us the value of the length of the perimeter and we will use the perimeter to approximate what type of shape this is.
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)#giving it resolution and then defining that this is a closed contour. This approximation array will have a certain amount of points and these points can determine whether its a square or any other shape.
            print(len(approx))
            #now lets create a bounding box
            x, y, w, h=cv2.boundingRect(approx)
            cv2.rectangle(imgContour, (x , y ), (x + w , y + h ), (0, 255, 0), 5) #2nd arg is initial points. 3rd argument is the final corner of the rectangle or the bounding box.after that color followed by width
            #now we are going to display our values so that we can easily see the number of points detected and the area detected.
            cv2.putText(imgContour, "Points: " + str(len(approx)), (x , y -42), cv2.FONT_HERSHEY_COMPLEX, .5,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "Area: " + str(int(area)), (x , y - 27), cv2.FONT_HERSHEY_COMPLEX, 0.5,
                        (0, 255, 0), 2)
            cv2.putText(imgContour, "fruit detected: Apple ", (x-15 , y - 12), cv2.FONT_HERSHEY_COMPLEX, 0.5,
                        (0, 255, 0), 2)
            
            
while True:
    frame = cv2.imread('apple (281).jpg')
    imgContour = frame.copy()
    imgBlur = cv2.medianBlur(frame, 7)#using kernel of 7 by 7
    
    b, g, r = cv2.split(imgBlur)
    blur = r-(0.5*b)-(0.5*g)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(70,70))
    #thresh1 = cv2.getTrackbarPos("Threshold 1", "Threshold Values")
    #thresh2 = cv2.getTrackbarPos("Threshold 2", "Threshold Values")
    _,th1 = cv2.threshold(blur, 51, 255, cv2.THRESH_BINARY)
    closing = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, kernel1)
    opening = cv2.morphologyEx(closing, cv2.MORPH_OPEN, kernel2)
    #opening_copy = np.uint8(opening)
    lap = cv2.Laplacian(opening, cv2.CV_64F, ksize=3)
    lap = np.uint8(np.absolute(lap))
    
    getContours(lap,imgContour)
    '''imgStack = stackImages(0.8,([frame,imgBlur,blur],
                               [frame,lap,imgContour]))'''
    
    #cv2.imshow('median blur',imgBlur)
    cv2.imshow('frame',frame)
    cv2.imshow('median blur',imgBlur)
    cv2.imshow('mask/blur',blur)
    cv2.imshow('laplace',lap)
    cv2.imshow('result',imgContour)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break
        
cv2.destroyAllWindows()
            



8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8


In [ ]:
# ignore

In [ ]:
# merging two codes- real time version (trial 1)
# disadvantage - lag/slow

In [ ]:
# object tracking 

In [5]:
import cv2

############### Tracker Types #####################

#tracker = cv2.TrackerBoosting_create()
#tracker = cv2.TrackerMIL_create()
#tracker = cv2.TrackerKCF_create()
#tracker = cv2.TrackerTLD_create()
#tracker = cv2.TrackerMedianFlow_create()
#tracker = cv2.TrackerCSRT_create()


########################################################

#refer to the blog for tracker description-https://ehsangazar.com/object-tracking-with-opencv-fd18ccdd7369

cap = cv2.VideoCapture(0)
#tracker=cv2.TrackerMOSSE_create() #speed is high but accuracy is low
tracker = cv2.TrackerCSRT_create()#accuracy is high but the speed is low

success, img=cap.read()
bbox=cv2.selectROI('Tracking',img,False)
tracker.init(img,bbox)

def drawBox(img,bbox):
    x, y, w, h = int(bbox[0]), int(bbox[1]), int(bbox[2]), int(bbox[3])
    cv2.rectangle(img, (x, y), ((x + w), (y + h)), (255, 0, 255), 3, 3 )#color
    cv2.putText(img, "Tracking", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
while True:
    timer = cv2.getTickCount()
    success, img=cap.read()
    
    success,bbox=tracker.update(img) #bbox is basically a touple
    print(bbox)
    
    if success:
        drawBox(img,bbox)
    else:
        cv2.putText(img, "Lost", (100, 75), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer);
    cv2.putText(img,str(int(fps)), (75, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.7,(0,0,255), 2);#scale, color, thickness
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()
    
    
    
    
    

(162.4857635498047, 244.76234436035156, 230.51998901367188, 212.16000366210938)
(168.76844787597656, 254.8934326171875, 217.22415161132812, 199.92311096191406)
(161.68997192382812, 245.82156372070312, 225.99998474121094, 207.99998474121094)
(166.75973510742188, 251.90298461914062, 217.22413635253906, 199.923095703125)
(171.8720703125, 253.9703369140625, 208.78907775878906, 192.15985107421875)
(185.2416229248047, 251.60040283203125, 200.68154907226562, 184.69805908203125)
(194.1891326904297, 251.72021484375, 200.68154907226562, 184.69805908203125)
(212.6831512451172, 249.726318359375, 200.68154907226562, 184.69805908203125)
(231.9871063232422, 249.07382202148438, 200.68154907226562, 184.69805908203125)
(257.1314697265625, 252.13296508789062, 196.74661254882812, 181.07652282714844)
(286.271728515625, 248.65560913085938, 200.68153381347656, 184.69805908203125)
(305.3644104003906, 250.56365966796875, 200.68153381347656, 184.69805908203125)
(318.1469421386719, 248.42266845703125, 208.789047

In [ ]:
# #feature detection

In [19]:
import cv2
import numpy as np
 
img1 = cv2.imread('apple (281).jpg',0)
#img2 = cv2.imread('ImagesTrain/Kinect.jpg',0)
img2 = cv2.imread('damaged_apple (263).jpg',0)
 
orb = cv2.ORB_create(nfeatures=1000)
 
kp1, des1 = orb.detectAndCompute(img1,None)
kp2, des2 = orb.detectAndCompute(img2,None)
# imgKp1 = cv2.drawKeypoints(img1,kp1,None)
# imgKp2 = cv2.drawKeypoints(img2,kp2,None)
print(des1.shape) #gives number of features and each feature is described in 32 values 
print(des1[0])

bf = cv2.BFMatcher()#brute force matcher
matches = bf.knnMatch(des1,des2,k=2)

#going through all the matches
good = []
for m,n in matches: #since we set k=2, we have 2 values that we can unpack
    if m.distance < 0.75*n.distance: #if distance between them is low then we are going to say that it is a good match. if dist between them is high then it is a bad match
        good.append([m])
print(len(good))
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)
 
# cv2.imshow('Kp1',imgKp1)
# cv2.imshow('Kp2',imgKp2)
cv2.imshow('img1',img1)
cv2.imshow('img2',img2)
cv2.imshow('img3',img3)
k = cv2.waitKey(1) & 0xFF
if k == 27:
    cv2.destroyAllWindows()
    

(549, 32)
[ 86 180 184 178 153 236  17  11 187  63 139  25 134 255  72  90 129 151
 109 145  58  75 154  18 217 246 123 255 128  64 135  83]
2


In [ ]:
# feature matching in real time

In [1]:
import cv2
import numpy as np
import os
 
current_path = os.getcwd()
orb = cv2.ORB_create(nfeatures=1000)
 
#### Import Images
images = []
classNames= []
folder = os.path.join(current_path, 'images')
myList = os.listdir(folder)
#print(myList)
print('Total Classes Detected', len(myList))
for cl in myList:
    imgCur = cv2.imread(f'{folder}/{cl}',0)# 0 is for gray scale
    images.append(imgCur)
    classNames.append(os.path.splitext(cl)[0])#storing without the file extension
print(classNames)

#now lets find all the descriptors of the given image

def findDes(images):
    desList=[]
    for img in images:
        kp,des = orb.detectAndCompute(img,None)
        desList.append(des)
    return desList

def findID(img, desList,thres=5):
    kp2,des2 = orb.detectAndCompute(img,None)
    bf = cv2.BFMatcher()
    matchList=[]
    finalVal = -1
    try:
        for des in desList:
            matches = bf.knnMatch(des, des2, k=2)
            good = []
            for m, n in matches:
                if m.distance < 0.75 * n.distance:
                    good.append([m])
            matchList.append(len(good))
    except:
        pass
    # print(matchList)
    if len(matchList)!=0:
        if max(matchList) > thres:
            finalVal = matchList.index(max(matchList)) #finding the index of the maximum value of matchList
    return finalVal

desList=findDes(images)
print(len(desList))

cap = cv2.VideoCapture(0)
 
while True:
 
    success, img2 = cap.read()
    imgOriginal = img2.copy()
    img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
 
    id = findID(img2,desList)
    if id != -1:
        cv2.putText(imgOriginal,classNames[id],(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,255),2)# scale, color and thickness
 
 
    cv2.imshow('img2',imgOriginal)
    k = cv2.waitKey(1) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

Total Classes Detected 5
['apple (184)', 'apple (191)', 'apple (281)', 'damaged_apple (263)', 'damaged_apple (28)']
5
